Ben Trevett 의 [Sequence to Sequence Learning with Neural Networks](https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb) 튜토리얼을 한글 데이터셋에 적용해보는 연습이다. 데이터셋은 [AI Hub 한국어-영어 번역 말뭉치](http://www.aihub.or.kr/aidata/87/download)를 이용한다.

이 모델에서는 [Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215) 논문에서 제시한 구조대로 간단한 모델을 만들어본다.

# 개요

가장 유명한 seq2seq 모델은 RNN을 이용한 인코더-디코더 모델이다. 입력 문장은 인코더를 통해 하나의 문맥 벡터로 변환되고, 이후 디코더에서 타겟 문장으로 변환된다.

![seq2seq](https://github.com/bentrevett/pytorch-seq2seq/raw/49cbdd39d934633ab69b7ff0cf4ef0da33a42e18/assets/seq2seq1.png)

문장의 처음에 `<sos>`, 마지막에 `<eos>` 토큰을 추가하여 입력으로 넣는다. 입력 문장의 각 단어 $x_t$에 대해 벡터 임베딩을 $e(x_t)$, 이전 타임의 히든 벡터를 $h_{t-1}$ 이라 하면 인코더는 다음과 같이 정의된다.

$$ h_t = \text{EncoderRNN}(e(X_t), h_{t-1}) $$

여기서 $X = \{ x_1,\ldots, x_T\}$, $x_1 = <sos>$ 등이다. 마지막 단어에 대한 히든 벡터 $h_T$를 문장의 벡터로 정의한다.

디코더는 타겟 문장 $(y_t)$의 임베딩 $d(y_t)$와 이전 히든 벡터 $s_{t-1}$에 대해 다음과 같이 정의된다($s_0 = h_T$).

$$ s_t = \text{DecoderRNN}(d(y_t), s_{t-1}) $$

마지막으로 디코더의 출력을 선형 레이어 $f$에 태워 각 단어의 최종 스코어 $\hat{y}_t = f(s_t)$ 를 만든다. 타겟 문장의 입력은 항상 `<sos>` 토큰으로 시작하지만, 그 이후의 벡터는 실제 타겟 단어 $y_t$를 입력을 넣을 수도 있고 아니면 이전 단계의 출력 $\hat{y}_t$를 넣을 수도 있다. 

이렇게 얻은 타겟 벡터 $\hat{Y} = \{ \hat{y}_1, \ldots, \hat{y}_T\}$ 를 실제 정답과 비교하여 손실을 계산하고 모델을 업데이트한다.

# 전처리

이후 사용할 모듈들을 불러오자.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, BucketIterator, TabularDataset

import spacy
from konlpy.tag import Mecab
import numpy as np

import random
import math
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

랜덤 시드를 고정하자.

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

토크나이저를 지정하자. 여기서 한글은 [KoNLPy](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)의 은전한닢, 영어는 [spaCy](https://spacy.io/)를 이용한다.

In [3]:
mecab = Mecab() # 한글
spacy_en = spacy.load('en') # 영어

토크나이저 함수를 만들자. 논문에 따르면 입력 문장은 역순으로 넣는 게 좋다고 하니 그대로 따르자.

In [4]:
def tokenize_ko(text):
    return [tok for tok in mecab.morphs(text)][::-1]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
tokenize_ko('한글은 너무 어렵당!')

['!', '당', '어렵', '너무', '은', '한글']

In [6]:
tokenize_en('Korean is too dificult for me!')

['Korean', 'is', 'too', 'dificult', 'for', 'me', '!']

TorchText의 `Field`를 이용해서 데이터 입력 포맷을 지정하자. `init_token`과 `eos_token` 인자를 이용해서 문장 처음과 마지막에 `<sos>`와 `<eos>` 토큰을 자동으로 추가할 수 있다.

In [7]:
SRC = Field(tokenize = tokenize_ko,
           init_token = '<sos>',
           eos_token = '<eos>',
           )

TRG = Field(tokenize = tokenize_en,
            init_token = '<sos>',
            eos_token = '<eos>',
            lower = True
           )

In [8]:
fields = {'ko': ('src',SRC), 'en': ('trg',TRG)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [9]:
!ls data

1.구어체.xlsx  3.문어체-뉴스.xlsx  train_data.csv
2.대화체.xlsx  test_data.csv	   valid_data.csv


In [10]:
train_data, test_data = TabularDataset.splits(
                            path = 'data',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields,  
)
valid_data = TabularDataset(path = 'data/valid_data.csv',
                            format = 'csv',
                            fields = fields,  )

불러온 데이터는 다음과 같은 형태이다.

In [11]:
vars(train_data[0]), vars(valid_data[0])

({'src': ['.', '요', '가', '안', '가', '이해', '이', '문장', '이', '님', '선생'],
  'trg': ['sir',
   ',',
   'i',
   'do',
   "n't",
   'understand',
   'this',
   'sentence',
   'here',
   '.']},
 {'src': ['.', '가요', '로', '기숙사', '자마자', '끝나', '가', '학교'],
  'trg': ['i',
   'go',
   'to',
   'dormitory',
   'as',
   'soon',
   'as',
   'i',
   'finished',
   'class',
   '.']})

길이를 확인해보자.

In [12]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 94463
Number of validation examples: 31688
Number of testing examples: 31822


이제 단어장을 만들자. `min_freq` 옵션을 이용하여 최소 2번 이상 등장하는 단어만 사용하도록 하자. 또한 단어장은 검증/테스트셋은 써서는 안된다.

In [13]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data, min_freq=2)

In [14]:
print(f"Unique tokens in source (ko) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (ko) vocabulary: 33345
Unique tokens in target (en) vocabulary: 24619


이터레이터를 만들자. 일반적인 `Iterator` 대신 `BucketIterator` 쓰면 입력/출력 배치 안에 있는 문장의 길이가 최대한 비슷하게 되어 패딩을 최소화하게 해준다.

In [15]:
BATCH_SIZE = 32

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.src),
    sort_within_batch = True,
    device = device)

In [16]:
next(iter(train_iterator)).src.shape

torch.Size([12, 32])

# Seq2seq 모델 생성

인코더, 디코더, 그리고 seq2seq 모델 순서대로 만들자.

## 인코더

인코더는 2레이어 LSTM을 사용하여 다음과 같이 정의한다.

$$ (h_t^1,c_t^1)= \text{EncoderLSTM}^1(e(x_t),(h_{t-1}^1,c_{t-1}^1)) $$

$$ (h_t^2,c_t^2)= \text{EncoderLSTM}^2(h_t^1,(h_{t-1}^2, c_{t-1}^2)) $$

![encoder](https://github.com/bentrevett/pytorch-seq2seq/raw/49cbdd39d934633ab69b7ff0cf4ef0da33a42e18/assets/seq2seq2.png)

인코더는 다음 인수들을 입력으로 받는다.

* `input_dim` : 입력 문장의 단어 갯수
* `emb_dim` : 임베딩 차원
* `hid_dim` : 히든 차원
* `n_layers` : RNN 계층수
* `dropout` : dropout 비율. 계층 사이에 적용된다.

출력은 `outputs`(최상위층 히든 벡터), `hidden`(각 층별 최종 히든 state를 쌓음), `cell`(각 층별 셀 state를 쌓은 벡터) 이다.

In [17]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        # src = [src_len, batch_size]
        embedded = self.dropout(self.embedding(src))
        
        # embedded = [src_len, batch_size, emb_dim]
        outputs, (hidden, cell) = self.rnn(embedded)
        
        # outputs = [src_len, batch_size, hid_dim * n_direction]
        #print("outputs shape : {}".format(outputs.shape))
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        #print("hidden shape : {}".format(hidden.shape))
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        #print("cell shape : {}".format(cell.shape))
        
        return hidden, cell

In [18]:
src = next(iter(train_iterator))
enc = Encoder(len(SRC.vocab), 200, 100, 2, 0.5)
enc.to('cuda')
enc(src.src)

(tensor([[[ 0.1226,  0.0346,  0.1716,  ...,  0.4877,  0.3778, -0.2108],
          [ 0.2642, -0.0190, -0.0653,  ...,  0.2019,  0.2856, -0.0675],
          [ 0.2633, -0.0312, -0.0503,  ..., -0.2938, -0.3479,  0.0081],
          ...,
          [ 0.0144,  0.1000,  0.1812,  ..., -0.0714, -0.3423, -0.0349],
          [ 0.0831,  0.0031, -0.2627,  ..., -0.4309,  0.4981, -0.1830],
          [ 0.0435,  0.0219,  0.0047,  ..., -0.3588,  0.4724, -0.1527]],
 
         [[-0.0057, -0.0647,  0.0599,  ..., -0.0166, -0.0216,  0.1694],
          [-0.0576, -0.0393,  0.1705,  ..., -0.0051,  0.0441,  0.1316],
          [-0.1214, -0.0838,  0.0903,  ...,  0.0542,  0.0354, -0.0320],
          ...,
          [-0.0762, -0.1770,  0.2006,  ...,  0.1002,  0.0772,  0.0884],
          [-0.0114, -0.0913,  0.1388,  ...,  0.1078,  0.0263,  0.0884],
          [-0.0319, -0.1424,  0.1051,  ...,  0.0573,  0.0237,  0.1356]]],
        device='cuda:0', grad_fn=<CudnnRnnBackward>),
 tensor([[[ 0.3097,  0.1161,  0.2274,  ...,  0.

# 디코더

디코더는 2층 LSTM 이며 다음 그림과 같은 구조를 가진다.

![decoder](https://github.com/bentrevett/pytorch-seq2seq/raw/6559ece8dcb41d2cb9cfe479c7442c8d6c0d90bb/assets/seq2seq3.png)

$$ (s_t^1,c_t^1)= \text{DecoderLSTM}^1(d(y_t),(s_{t-1}^1,c_{t-1}^1)) $$

$$ (s_t^2,c_t^2)= \text{DecoderLSTM}^2(s_t^1,(s_{t-1}^2, c_{t-1}^2)) $$

초기값은 인코더의 출력으로 다음과 같이 정의된다.

$$ (s_0^l, c_0^l) = z^l = (h_T^l, c_T^l) $$

가장 윗층의 히든 스테이트 $s_t^L$를 선형 계층 $f$에 통과시켜 다음 토큰의 예측값 $\hat{y}_{t+1}$을 얻는다.

순전파 과정에서 디코더는 한번에 하나씩의 토큰만 처리한다. 

In [19]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim , n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        # input = [batch_size]
        # hidden = [n_layers * n_direction, batch_size, hid_dim]
        # cell = [n_layers * n_direction, batch_size, hid_dim]
        
        input = input.unsqueeze(0)
        # input = [1, batch_size]
        
        embedded = self.dropout(self.embedding(input))
        # embedded = [1, batch_size, emb_dim]
        
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # output = [1, batch size, hid dim * n directions]
        # hidden = [n layers * n directions, batch size, hid dim]
        # cell = [n layers * n directions, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        # prediction = [batch_size, output_dim]
        
        return prediction, hidden, cell

# Seq2seq

다음을 수행하는 seq2seq 모델을 만들자.

* 원 문장을 입력으로 받고
* 인코더를 이용하여 문맥 벡터를 만든 후
* 디코더를 이용하여 결과 문장을 예측한다.

![seq2seq](https://github.com/bentrevett/pytorch-seq2seq/raw/6559ece8dcb41d2cb9cfe479c7442c8d6c0d90bb/assets/seq2seq4.png)

순전파 과정에서 원 문장과 타겟 문장과 `teacher-forcing rate`를 입력으로 받는다. `teacher-forcing rate`는 훈련 과정에서 다음 토큰의 입력을 실제 타겟 문장의 토큰으로 할지, 아니면 이전 토큰의 결과값으로 할지 비율을 결정한다. 

In [20]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
        "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
        "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        """
        src = [src_len, batch_size]
        trg = [trg_len, batch_size]
        """
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        # 디코더 출력값을 저장할 텐서
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        # 인코더의 마지막 히든 스테이트는 디코더의 최초 히든 스테이트
        hidden, cell = self.encoder(src)
        
        # 디코더의 입력의 처음은 <sos> 토큰
        input = trg[0,:]
        
        for t in range(1, trg_len):
            # 인풋 토큰, 이전 히든/셀 스테이트를 입력으로 넣고
            # 아웃풋 텐서, 새로운 히든/셀 스테이트를 출력
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            # outputs에 저장 (output = [batch_size, output_dim])
            outputs[t] = output
            
            # teacher forcing 쓸지 말지
            teacher_force = random.random() < teacher_forcing_ratio
            
            # 출력중 최고값
            top1 = output.argmax(1)
            
            # teacher_forcing=True 이면 groud truth,
            # 아니면 이전 예측값을 다음 입력으로 넣음
            input = trg[t] if teacher_force else top1
            
        return outputs
            

# Seq2seq 모델 훈련

하이퍼 파라미터들을 정하자.

In [21]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

모델의 초기값은 $\mathfrak{U}(-0.08,0.08)$을 따르도록 한다. 이때 `apply` 메서드를 적용하는데, 이 메서드는 입력으로 받은 함수를 각 모듈과 서브모듈에 적용한다.

In [22]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(33345, 128)
    (rnn): LSTM(128, 256, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(24619, 128)
    (rnn): LSTM(128, 256, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=256, out_features=24619, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

파라미터 갯수는?

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 15,589,675 trainable parameters


옵티마이저는 Adam을 쓴다.

In [24]:
optimizer = optim.Adam(model.parameters())

손실함수는 `CrossEntropyLoss` 를 쓴다. 다만 `<pad>` 토큰은 무시하도록 `igonre_index` 옵션을 넣어준다.

In [25]:
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

훈련 과정을 정의하자. 이때 trg 벡터의 첫번째 토큰은 항상 `<sos>` 토큰이고, 이 토큰은 학습시키지 않는다. 이때 주의할 점은 다음과 같다.

* 손실 함수는 2차원 입력과 1차원 타겟을 다루게 되어 있으므로 입력을 `view()` 메서드를 이용해서 변환해주어야 한다.
* Gradient exploding 현상을 방지하기 위해 gradient clapping을 적용한다.

In [26]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        output = model(src, trg)
        
        # trg = [trg_len, batch_size]
        # output = [trg_len, batch_size, output_dim]
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim) # <sos> 토큰 제외
        trg = trg[1:].view(-1)
        
        # trg = [(trg_len - 1) * batch_size]
        # output = [(trg_len - 1) * batch_size, output_dim]
        
        loss = criterion(output, trg)
        loss.backward()
        
        # gradient clapping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

평가 과정에서는 `teacher_forcing`을 꺼주어야 한다는 점에 주의!

In [27]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg
            
            output = model(src, trg, 0) # teacher forcing 제거
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

훈련 시간 측정 함수를 만들자.

In [28]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

훈련을 시켜보자. 각 에폭마다 성능이 좋아지면 모델의 파라미터를 저장하도록 한다. 그리고 loss와 perplexity(=exp(loss))를 출력하도록 한다.

In [ ]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')    

Epoch: 01 | Time: 8m 10s
	Train Loss: 5.726 | Train PPL: 306.708
	 Val. Loss: 5.802 |  Val. PPL: 331.111
Epoch: 02 | Time: 8m 9s
	Train Loss: 5.202 | Train PPL: 181.621
	 Val. Loss: 5.618 |  Val. PPL: 275.344
Epoch: 03 | Time: 8m 9s
	Train Loss: 4.910 | Train PPL: 135.577
	 Val. Loss: 5.420 |  Val. PPL: 225.901
Epoch: 04 | Time: 8m 9s
	Train Loss: 4.685 | Train PPL: 108.261
	 Val. Loss: 5.296 |  Val. PPL: 199.561
Epoch: 05 | Time: 8m 7s
	Train Loss: 4.509 | Train PPL:  90.793
	 Val. Loss: 5.231 |  Val. PPL: 186.899
Epoch: 06 | Time: 8m 10s
	Train Loss: 4.380 | Train PPL:  79.827
	 Val. Loss: 5.167 |  Val. PPL: 175.458
Epoch: 07 | Time: 8m 7s
	Train Loss: 4.265 | Train PPL:  71.140
	 Val. Loss: 5.130 |  Val. PPL: 169.061
Epoch: 08 | Time: 8m 9s
	Train Loss: 4.166 | Train PPL:  64.439
	 Val. Loss: 5.100 |  Val. PPL: 164.011


테스트셋에 확인해보자!

In [31]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 5.086 | Test PPL: 161.698 |


추가 훈련을 시켜보자.

In [32]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
        
    print(f'Epoch: {epoch+9:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')    

Epoch: 09 | Time: 8m 8s
	Train Loss: 3.939 | Train PPL:  51.361
	 Val. Loss: 5.048 |  Val. PPL: 155.668
Epoch: 10 | Time: 8m 8s
	Train Loss: 3.887 | Train PPL:  48.788
	 Val. Loss: 5.027 |  Val. PPL: 152.471
Epoch: 11 | Time: 8m 9s
	Train Loss: 3.835 | Train PPL:  46.285
	 Val. Loss: 5.011 |  Val. PPL: 149.985
Epoch: 12 | Time: 8m 10s
	Train Loss: 3.793 | Train PPL:  44.382
	 Val. Loss: 5.030 |  Val. PPL: 152.949
Epoch: 13 | Time: 8m 7s
	Train Loss: 3.747 | Train PPL:  42.409
	 Val. Loss: 5.002 |  Val. PPL: 148.685
Epoch: 14 | Time: 8m 8s
	Train Loss: 3.711 | Train PPL:  40.881
	 Val. Loss: 5.049 |  Val. PPL: 155.944
Epoch: 15 | Time: 8m 9s
	Train Loss: 3.671 | Train PPL:  39.296
	 Val. Loss: 5.036 |  Val. PPL: 153.832
Epoch: 16 | Time: 8m 11s
	Train Loss: 3.635 | Train PPL:  37.902
	 Val. Loss: 5.025 |  Val. PPL: 152.102
Epoch: 17 | Time: 8m 11s
	Train Loss: 3.602 | Train PPL:  36.675
	 Val. Loss: 5.007 |  Val. PPL: 149.517
Epoch: 18 | Time: 8m 11s
	Train Loss: 3.572 | Train PPL:  35.

# Inference

추론 함수를 구현한 후 실제 번역을 돌려보자.

In [100]:
def predict_sentiment(model, sentence):
    model.eval()
    
    # 한글 문장을 역순으로 토크나이징
    tokenized = [tok for tok in reversed(mecab.morphs(sentence))]
    #print(tokenized)
    
    # 문장 앞뒤에 <sos>, <eos> 토큰 추가
    indexed = [SRC.vocab.stoi[SRC.init_token]]+[SRC.vocab.stoi[t] for t in tokenized]+[SRC.vocab.stoi[SRC.eos_token]]
    #print(indexed)
    
    # LongTensor 변환
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1) # 배치 
    #print(tensor)
    
    # TRG 문장은 처음에만 <sos> 토큰을 넣고 나머진 0 으로 입력
    zero_trg = torch.LongTensor([[TRG.vocab.stoi[TRG.init_token]]+[0 for _ in range(100)]]).t().to(device)
    #print(zero_trg.shape)
    outputs = model(tensor, zero_trg, 0)
    
    # 모델 출력값으로부터 번역 문장 생성
    # <eos> 토큰을 만나면 거기에서 종료
    res = []
    for i in range(1,outputs.shape[0]):
        ind = outputs[i].argmax(1)
        if ind == TRG.vocab.stoi[TRG.eos_token]:
            break
        res.append(TRG.vocab.itos[ind])
    return ' '.join(res)

In [101]:
predict_sentiment(model, '밥은 먹고 다니냐?')

'do you eat eat rice ?'

In [107]:
predict_sentiment(model, '오늘 하늘은 하루종일 맑다.')

"today 's day is clean today ."

In [108]:
sent = """
생중계에서 모든 과정을 총지휘하는 연출가에 가깝다.
"""
predict_sentiment(model, sent)

"it 's a program that goes to the the ."

In [109]:
sent = """
무대에 오르는 배우 숫자에 따라 카메라 수도 변한다. 
"""
predict_sentiment(model, sent)

'the change the the the the the the the the . .'

In [110]:
sent = """
전국 곳곳에서 최선을 다해 치료해 주시는 의료진 선생님들 힘내세요 화이팅!!!!
"""
predict_sentiment(model, sent)

'welcome your best for the health of the health !'

In [112]:
sent = """
오늘 결혼식장에 다녀왔다.
"""
predict_sentiment(model, sent)

'i went to the restaurant today .'

In [113]:
sent = """
청소하기 너무 귀찮아!
"""
predict_sentiment(model, sent)

"it 's too hard to clean your hands !"

이걸로는 안되겠다! ㅋㅋㅋ